<a href="https://colab.research.google.com/github/pksj/IIITA_2nd_SEM/blob/main/NER_using_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import csv  
# import sklearn
!pip install sklearn-crfsuite
import sklearn_crfsuite
import math, string, re
from sklearn.metrics import accuracy_score
from sklearn_crfsuite import metrics

#Function to create dataset

In [ ]:
def create_dataset(file_path, data_type):

    file1 = open(file_path, 'r')
    all_lines = file1.readlines()
    # print("all_lines = ", all_lines)
    all_rows = []

    id = 1

    for line in all_lines:
        t = line.split("\t")
        # print("t = ",t)
        
        row = []
        if(len(t) == 1):
            id = 1
            row.append(" ")
            row.append(" ")
            row.append(" ")
        else:
            row.append(id)
            row.append(t[0])
            row.append(t[1].strip('\n'))
            id += 1
        all_rows.append(row)
    
    fields = ['ID', 'Word', 'tag'] 

    if data_type == 'train':
        filename = "NER_train.csv"
    else:
        filename = "NER_test.csv"


    with open(filename, 'w') as csvfile: 

        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(fields) 
        csvwriter.writerows(all_rows)



In [ ]:
create_dataset('/content/test', 'test')
create_dataset('/content/train', 'train')

data = {}

data['train'] = pd.read_csv('/content/NER_test.csv')
data['test'] = pd.read_csv('/content/NER_train.csv')

# print(data['train'], data['test'], sep = '\n\n')
# print(data['train']['ID'].dtype)


#Fucntion to extract features

In [ ]:
def word_to_features(sentence, i):
    
    word = sentence[i][0]

    features = {
        'word': word,
        'len(word)': len(word),
        # 'word[:4]': word[:4],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        # 'word[-4:]': word[-4:],
        'word.lower()': word.lower(),
        # 'word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', word.lower()),
        'word.ispunctuation': (word in string.punctuation),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sentence[i-1][0]
        features.update({
            '-1:word': word1,
            '-1:len(word)': len(word1),
            '-1:word.lower()': word1.lower(),
            # '-1:word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', word1.lower()),
            '-1:word[:3]': word1[:3],
            '-1:word[:2]': word1[:2],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.ispunctuation': (word1 in string.punctuation),
        })

    else:
        features['BOS'] = True

    if i > 1:
        word2 = sentence[i-2][0]
        features.update({
            '-2:word': word2,
            '-2:len(word)': len(word2),
            '-2:word.lower()': word2.lower(),
            '-2:word[:3]': word2[:3],
            '-2:word[:2]': word2[:2],
            '-2:word[-3:]': word2[-3:],
            '-2:word[-2:]': word2[-2:],
            '-2:word.isdigit()': word2.isdigit(),
            '-2:word.ispunctuation': (word2 in string.punctuation),
        })

    if i < len(sentence)-1:
        word1 = sentence[i+1][0]
        features.update({
            '+1:word': word1,
            '+1:len(word)': len(word1),
            '+1:word.lower()': word1.lower(),
            '+1:word[:3]': word1[:3],
            '+1:word[:2]': word1[:2],
            '+1:word[-3:]': word1[-3:],
            '+1:word[-2:]': word1[-2:],
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.ispunctuation': (word1 in string.punctuation),
        })

    else:
        features['EOS'] = True

    if i < len(sentence) - 2:
        word2 = sentence[i+2][0]
        features.update({
            '+2:word': word2,
            '+2:len(word)': len(word2),
            '+2:word.lower()': word2.lower(),
            # '+2:word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', word2.lower()),
            '+2:word[:3]': word2[:3],
            '+2:word[:2]': word2[:2],
            '+2:word[-3:]': word2[-3:],
            '+2:word[-2:]': word2[-2:],
            '+2:word.isdigit()': word2.isdigit(),
            '+2:word.ispunctuation': (word2 in string.punctuation),
        })
    print("Sentence = ",sentence)
    print("Index = ",i)
    print("Features = ",features)
    return features


def sentence_to_features(sentence):
    return [word_to_features(sentence, i) for i in range(len(sentence))]

def sentence_to_labels(sentence):
    return [word[1] for word in sentence]

# def sentence_to_tokens(sentence):
#     return [word[0] for word in sentence]


In [ ]:
def format_data(csv_data):
    
    sentences = []
    
    for i in range(len(csv_data)):
       
        if csv_data.iloc[i, 0] == ' ':
            continue
        elif int(csv_data.iloc[i, 0]) == 1:
            sentences.append([[csv_data.iloc[i, 1], csv_data.iloc[i, 2]]])
        else:
            sentences[-1].append([csv_data.iloc[i, 1], csv_data.iloc[i, 2]])
    
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if type(word[0]) != str:
                # print(sentence[i])
                del sentence[i]
    
    # print(sentences)
    return sentences

#Extracting features from all the sentences

In [ ]:

train_sentences = format_data(data['train'])
test_sentences = format_data(data['test'])
print(train_sentences[0])

Xtrain = [sentence_to_features(s) for s in train_sentences]
ytrain = [sentence_to_labels(s) for s in train_sentences]
print(Xtrain[0:5])
print("\n\n")
print(ytrain[0:5])

Xtest = [sentence_to_features(s) for s in test_sentences]
ytest = [sentence_to_labels(s) for s in test_sentences]

Streaming output truncated to the last 5000 lines.
Index =  3
Features =  {'word': 'last', 'len(word)': 4, 'word[:3]': 'las', 'word[:2]': 'la', 'word[-3:]': 'ast', 'word[-2:]': 'st', 'word.lower()': 'last', 'word.ispunctuation': False, 'word.isdigit()': False, '-1:word': 'my', '-1:len(word)': 2, '-1:word.lower()': 'my', '-1:word[:3]': 'my', '-1:word[:2]': 'my', '-1:word[-3:]': 'my', '-1:word[-2:]': 'my', '-1:word.isdigit()': False, '-1:word.ispunctuation': False, '-2:word': 'is', '-2:len(word)': 2, '-2:word.lower()': 'is', '-2:word[:3]': 'is', '-2:word[:2]': 'is', '-2:word[-3:]': 'is', '-2:word[-2:]': 'is', '-2:word.isdigit()': False, '-2:word.ispunctuation': False, '+1:word': 'day', '+1:len(word)': 3, '+1:word.lower()': 'day', '+1:word[:3]': 'day', '+1:word[:2]': 'da', '+1:word[-3:]': 'day', '+1:word[-2:]': 'ay', '+1:word.isdigit()': False, '+1:word.ispunctuation': False, '+2:word': 'at', '+2:len(word)': 2, '+2:word.lower()': 'at', '+2:word[:3]': 'at', '+2:word[:2]': 'at', '+2:word[-3

#Training the model using CRF

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    c1 = 0.25,
    c2 = 0.3,
    max_iterations = 100,
    all_possible_transitions=True
)
crf.fit(Xtrain, ytrain)                  

#Obtaining metrics such as accuracy, F1 score on the train set

In [ ]:

labels = list(crf.classes_)
# print(labels)

ypred = crf.predict(Xtrain)
print('F1 score on the train set = {}\n'.format(metrics.flat_f1_score(ytrain, ypred, average='weighted', labels=labels)))
print('Accuracy on the train set = {}\n'.format(metrics.flat_accuracy_score(ytrain, ypred)))

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print('Train set classification report: \n\n{}'.format(metrics.flat_classification_report(
    ytrain, ypred, labels=sorted_labels, digits=3)))

F1 score on the train set = 0.9986538556779353

Accuracy on the train set = 0.9986586942469295

Train set classification report: 

               precision    recall  f1-score   support

            O      0.999     1.000     0.999     55925
    B-company      1.000     0.984     0.992       621
    I-company      1.000     0.992     0.996       265
   B-facility      0.988     0.996     0.992       253
   I-facility      0.995     1.000     0.997       366
    B-geo-loc      0.997     0.992     0.994       882
    I-geo-loc      0.991     0.995     0.993       219
      B-movie      1.000     0.941     0.970        34
      I-movie      1.000     1.000     1.000        48
B-musicartist      1.000     0.984     0.992       191
I-musicartist      0.993     1.000     0.996       140
      B-other      0.990     0.978     0.984       584
      I-other      0.991     0.996     0.994       556
     B-person      1.000     0.973     0.986       482
     I-person      1.000     1.000     1.00

#Obtaining metrics such as accuracy, F1 score on the test set

In [23]:
ypred = crf.predict(Xtest)
print('F1 score on the test set = {}\n'.format(metrics.flat_f1_score(ytest, ypred,
                      average='weighted', labels=labels)))
print('Accuracy on the test set = {}\n'.format(metrics.flat_accuracy_score(ytest, ypred)))

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print('Test set classification report: \n\n{}'.format(metrics.flat_classification_report(
    ytest, ypred, labels=sorted_labels, digits=3)))

F1 score on the test set = 0.9345279825871127

Accuracy on the test set = 0.9477070735328929



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test set classification report: 

               precision    recall  f1-score   support

            O      0.959     0.993     0.976     44007
    B-company      0.606     0.117     0.196       171
    I-company      0.095     0.056     0.070        36
   B-facility      0.585     0.231     0.331       104
   I-facility      0.536     0.286     0.373       105
    B-geo-loc      0.765     0.272     0.401       276
    I-geo-loc      0.615     0.163     0.258        49
      B-movie      0.000     0.000     0.000        34
      I-movie      0.000     0.000     0.000        46
B-musicartist      0.040     0.018     0.025        55
I-musicartist      0.067     0.033     0.044        61
      B-other      0.268     0.084     0.128       225
      I-other      0.235     0.122     0.160       320
     B-person      0.656     0.140     0.231       449
     I-person      0.565     0.181     0.275       215
    B-product      0.111     0.031     0.048        97
    I-product      0.008     0

In [24]:
from collections import Counter

def print_transitions(transition_features):
    for (label_from, label_to), weight in transition_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top 10 likely transitions - \n")
print_transitions(Counter(crf.transition_features_).most_common(10))

print("\nTop 10 unlikely transitions - \n")
print_transitions(Counter(crf.transition_features_).most_common()[-10:])

Top 10 likely transitions - 

B-facility -> I-facility 4.984923
B-musicartist -> I-musicartist 4.978080
I-tvshow -> I-tvshow 4.729673
B-person -> I-person 4.681502
B-product -> I-product 4.463975
B-sportsteam -> I-sportsteam 4.446443
B-movie -> I-movie 4.427235
I-movie -> I-movie 4.412558
I-product -> I-product 4.374495
I-musicartist -> I-musicartist 4.318085

Top 10 unlikely transitions - 

O      -> I-movie -1.855796
O      -> I-tvshow -2.047110
O      -> I-sportsteam -2.077737
O      -> I-musicartist -2.193535
O      -> I-geo-loc -2.317300
O      -> I-facility -2.718555
O      -> I-company -2.792835
O      -> I-person -2.805819
O      -> I-product -3.203260
O      -> I-other -3.365892


In [25]:
print(1 == 1.01
      )

False
False
